### Define functions for data readout

In [1]:
def readDataFromCsv(filename):
    """Read data from .csv file into an array.
    Output: a numpy array"""
    import numpy as np
    import csv
    data = np.genfromtxt(filename, delimiter = ',', dtype = None)
    return data

In [2]:
def splitTrainTest(dataXY,partVec):
    """Split input table into two tables with rows defined according to partVec - array that defines partitioning.
    Return a dictionary with split train and test data"""
    sel1 = partVec==1 # train
    sel2 = partVec==2 # test
    dataXY1 = dataXY[sel1]
    dataXY2 = dataXY[sel2]
    dataSplit = {'trainData':dataXY1, 'testData':dataXY2}
    return dataSplit

In [3]:
def augmentData(dataXY, outcomeName, numberPerClass):
    """Augment tabular and image data for training.
    
    Produces equal number of augmented images for each class specified by the outcomeName.
    numberPerClass is the TOTAL number of rows PER class (this x2 in total)
    
    Assumes that the outcome is either '1' or '2'
    Output: first class '1' augmentations, then class '2' augmentations (must randomize externally)
    """
    totalClass1 = sum(dataXY[:][outcomeName] == 1)
    totalClass2 = sum(dataXY[:][outcomeName] == 2)
    
    # determine the number of augmentations required for each class
    numAug = [int(np.ceil(float(numberPerClass)/totalClass1)), int(np.ceil(float(numberPerClass)/totalClass2))]
    dataXYaug1 = np.zeros((totalClass1*numAug[0],),dtype = dataXY.dtype)
    dataXYaug2 = np.zeros((totalClass2*numAug[1],),dtype = dataXY.dtype)
    # generate class one augmentations
    addCount = 0
    for r in range(dataXY.size):
        if dataXY[r][outcomeName] == 1:
            newRow = dataXY[r]
            for nr in range(numAug[0]):
                # replicate the row but change the image number
                # add row to the new table
                dataXYaug1[addCount] = newRow 
                st = str(dataXYaug1[addCount]['IMAGEID_BL']) + '-' + str(nr+1)
                dataXYaug1[addCount]['IMAGEID_BL'] = np.str_(st)
                addCount = addCount + 1
                
    # generate class two augmentations
    addCount = 0
    for r in range(dataXY.size):
        if dataXY[r][outcomeName] == 2:
            newRow = dataXY[r]
            for nr in range(numAug[1]):
                # replicate the row but change the image number
                # add row to the new table
                dataXYaug2[addCount] = newRow
                st = str(dataXYaug2[addCount]['IMAGEID_BL']) + '-' + str(nr+1)
                dataXYaug2[addCount]['IMAGEID_BL'] = np.str_(st)
                addCount = addCount + 1
                
    # crop the new tables to the specified size
    dataXYaugComb = np.concatenate((dataXYaug1[0:numberPerClass],dataXYaug2[0:numberPerClass]))
    return dataXYaugComb

In [4]:
def dataStream(dataTable, dataDef, batchSize, batchNumber):
    # Read and output data as specified, batch size is number of items per iteration
    # Start batch index is zero
    
    # Also loads metadata that is added directly to the dense layer
    
    # must implement ability to feed multiple epochs
    xVariableNames = dataDef['xVarNames']
    yVariableName = dataDef['yVarName']
    numXvariables = len(xVariableNames)
    imageDir = dataDef['imageDir']
    imWidth = dataDef['imageSize'][0]
    imHeight = dataDef['imageSize'][1]
    imDepth = dataDef['imageSize'][2]
 
    beginInd = batchSize * batchNumber
    endInd = beginInd + batchSize
    
    image_data = np.array([], np.float32)
    meta_data = np.zeros((batchSize, numXvariables))
    y_data = np.zeros((batchSize, 1))
    # check for out-of-bounds error
    if (beginInd+1>len(dataTable)) or (endInd>len(dataTable)):
        raise ValueError('Requested batch is outside of the data table bounds')
    
    count = 0
    for i in range(beginInd,endInd):
        # load image data
        imageFilename = imageDir + dataTable[i]['IMAGEID_BL'] + '.mat'
        #print('Reading ' + imageFilename)
        FA_org = sio.loadmat(imageFilename)
        FA_data = FA_org['img'] #numpy.ndarray [32 48 16]
        image = FA_data.copy()
        image_data = np.append(image_data, np.asarray(image, dtype='float32')) # (image.data, dtype='float32')
        # load x data 
        meta_data[count] = list(dataTable[xVariableNames][i])
        #meta_data[1] = list(dataXY[['PUTAMEN_L','PUTAMEN_R','CAUDATE_L']][1])

        y_data[count] = dataTable[i][yVariableName]
        count += 1
        
    image_data_rs = image_data.reshape(batchSize, imHeight * imWidth * imDepth)
    
    xData = {'xDataImage':image_data_rs, 'xDataMeta':meta_data}
    yData = {'yData':y_data}
    return xData,yData

In [5]:
def oneHotEncoding(classData):
    # Convert class labels to one-hot encoding
    # classData is a Numpy column-array
    classes = np.unique(classData)
    numOfClasses = np.size(classes)
    classDataOH = np.zeros([np.size(classData),numOfClasses])
    for i in range(np.size(classData)):
        index = np.where(classes==classData[i])
        classDataOH[i,index] = 1
    return classDataOH

### Main function that performs CNN initialization, training and testing

In [6]:
def cnnTrainTest(dataTrain, dataTest, dataDef, cnnDef, outputDir):
    # Train and test CNN using specified data and parameters
    # dataDef: dictionary with data parameters
    # cnnDef: dictionary with CNN parameters
    
    imageSize = dataDef['imageSize']
    classes = np.unique(dataTest[:][dataDef['yVarName']])
    numClasses = np.size(classes)

    numberOfMetaVariables = len(dataDef['xVarNames'])
    
    xImage = tf.placeholder(tf.float32, shape=[None, np.prod(imageSize)]) # 32*48*16
    xMeta = tf.placeholder(tf.float32, shape=[None, numberOfMetaVariables]) # goes into the fully-connected layer
    y_ = tf.placeholder(tf.float32, shape=[None, numClasses]) # one-hot encoding
    
    # Set random seed
    tf.set_random_seed(12345)
    
    # Init weights with small amount of noise for symmetry breaking
    
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1, seed=0)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)
    
    def conv3d(x, W):
        return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME') # Computes a 3-D convolution given 5-D input and filter tensors

    def max_pool_2x2x2(x): # Must be one of the following types: float32. Shape [batch, depth, rows, cols, channels] tensor to pool over.
        return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

    # Structure implementation
    
    # First layer
    xImageRS = tf.reshape(xImage, [-1, 16, 32, 48, 1]) # BATCH, DEPTH, HEIGHT, WIDTH, CHANNELS
    W_conv1 = weight_variable([5, 5, 5, 1, 32]) # use just one (original) channel
    b_conv1 = bias_variable([32])
    
    h_conv1 = tf.nn.relu(conv3d(xImageRS, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2x2(h_conv1) # should have size 8, 16, 24, 32

    # Second layer
    W_conv2 = weight_variable([5, 5, 5, 32, 64]) # use all 32 channels in convolution
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv3d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2x2(h_conv2) # should have size 4, 8, 12, 64

    # Third layer
    W_conv3 = weight_variable([5, 5, 5, 64, 64]) # use all 64 channels in convolution
    b_conv3 = bias_variable([64])

    h_conv3 = tf.nn.relu(conv3d(h_pool2, W_conv3) + b_conv3)
    h_pool3 = max_pool_2x2x2(h_conv3) # should have size 2, 4, 6, 64
    h_pool3_flat = tf.reshape(h_pool3, [-1, 2 * 4 * 6 * 64])

    # Dropout
    keep_prob = tf.placeholder(tf.float32)
    h_pool3_flat_drop = tf.nn.dropout(h_pool3_flat, keep_prob)
    
    # Fully-connected
    W_fc1 = weight_variable([2 * 4 * 6 * 64, 16])
    b_fc1 = bias_variable([16])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat_drop, W_fc1) + b_fc1)
    
    # Add auxiliary data to the fully connected layer
    h_meta = tf.reshape(xMeta,[-1, numberOfMetaVariables])
    h_combined = tf.concat((h_fc1, h_meta), axis=1)
    
    # Readout layer
    W_fc2 = weight_variable([16+numberOfMetaVariables, 2])
    b_fc2 = bias_variable([2])
    y_conv = tf.matmul(h_combined, W_fc2) + b_fc2
    
    #loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_1, logits=y_conv)
    #loss = tf.losses.softmax_cross_entropy(onehot_labels=y_, logits=y_conv)
    #loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=y_, logits=y_conv)
    #loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    
    classWeights = tf.cast(tf.argmin(y_, 1),tf.float32)*100 + 1
    loss = tf.losses.softmax_cross_entropy(onehot_labels=y_, logits=y_conv, weights=classWeights) # for probabilistic classes
    
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
    
    probabilities = tf.nn.softmax(y_conv,name="softmax_tensor")
    
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    numberOfIterations = cnnDef['iterations']
    batchSize = cnnDef['batchSize']
    numberOfTrainingSamples = len(dataTrain)
    numberOfTestSamples = len(dataTest)
    numberOfEpochs = float(batchSize * numberOfIterations) / numberOfTrainingSamples
    iterationsPerEpoch = numberOfTrainingSamples/batchSize
    
    print 'Total train size %i, batch size %i, iterations per epoch %i' % (numberOfTrainingSamples,batchSize,iterationsPerEpoch)
    print 'Training with %i iterations, %f epochs...' % (numberOfIterations,numberOfEpochs)
    
    lossTrain = np.array([])
    lossTest = np.array([])
    accTrain = np.array([])
    accTest = np.array([])
    probTest = []
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        #print 'Reading testing batch'
        xD, yD = dataStream(dataTest, dataDef, numberOfTestSamples, 0) # bn is the iteration/batch number starts with 0 
        x_test_im  = xD['xDataImage']
        x_test_meta  = xD['xDataMeta']
        y_test = oneHotEncoding(yD['yData']) # convert output to one-hot encoding
        #y_test = tf.one_hot(yD['yData'],2)
        
        for it in range(numberOfIterations):
            timeStart = time.time()
            
            keep_prob_dynamic = 1.0
            #print 'Reading training batch %i' % bn
            
            # determine appropriate batch number
            bn = it % iterationsPerEpoch
            xD, yD = dataStream(dataTrain, dataDef, batchSize, bn) # bn is the iteration/batch number starts with 0 
            x_batch_im  = xD['xDataImage']
            x_batch_meta = xD['xDataMeta']
            y_batch = oneHotEncoding(yD['yData']) # convert output to one-hot encoding

            #y_batch = tf.one_hot(yD['yData'],2)
            #print 'Batch reading complete in %f s' % ((time.time()-timeStart))
            
            if it % 10 == 0:
                #print('Iteration %d, training accuracy %g, test accuracy %g' % (it, accTrain[-1], accTest[-1]))
                lossTrain = np.append(lossTrain,loss.eval(feed_dict={xImage: x_batch_im,
                                                                     xMeta: x_batch_meta,
                                                                     y_: y_batch, 
                                                                     keep_prob: keep_prob_dynamic}))
                accTrain = np.append(accTrain, acc.eval(feed_dict={xImage: x_batch_im, 
                                                                   xMeta:x_batch_meta, 
                                                                   y_: y_batch, 
                                                                   keep_prob: keep_prob_dynamic}))
                lossTest = np.append(lossTest,loss.eval(feed_dict={xImage: x_test_im,
                                                                   xMeta: x_test_meta,
                                                                   y_: y_test, 
                                                                   keep_prob: keep_prob_dynamic}))
                accTest = np.append(accTest, acc.eval(feed_dict={xImage: x_test_im,
                                                                 xMeta: x_test_meta,
                                                                 y_: y_test, 
                                                                 keep_prob: keep_prob_dynamic}))
                probTest = probTest + [probabilities.eval(feed_dict={xImage: x_test_im,
                                                              xMeta: x_test_meta,
                                                              y_: y_test, 
                                                              keep_prob: keep_prob_dynamic})]
                print('Iteration %d, train loss %g, test loss %g' % (it, lossTrain[-1], lossTest[-1]))
                classIndex = np.argmax(y_batch,axis=1)
                print('Class ratio %g' % np.mean(classIndex))
            # Training happens here
            train_step.run(feed_dict={xImage: x_batch_im,
                                      xMeta: x_batch_meta,
                                      y_: y_batch, 
                                      keep_prob: keep_prob_dynamic})
            
        outputDict = {'lossTrain':lossTrain, 
                      'lossTest':lossTest, 
                      'accTrain':accTrain, 
                      'accTest':accTest,
                      'probTest':probTest}
        return outputDict
            # Return:
            # Loss train, loss test
            # Accuracy train, accuracy test
            # Predicted probabilities for y test


# Script to perform cross-validated CNN training/testing

In [7]:
import numpy as np
import tensorflow as tf
import scipy.io as sio
import time
import sys
import os

if sys.platform == 'darwin':
    homeDir = '/Users/'
else:
    homeDir = '/home/'
    
# Read data
dataFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/data-cog2y.csv'
variableFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/names-cog2y.csv'
partitionsFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/part-cog2y.csv'

dataXYtmp = readDataFromCsv(dataFilename)
variableNames = readDataFromCsv(variableFilename)
partitions = readDataFromCsv(partitionsFilename) # 1's are for train, 2's are for test

# Check - print variable names
print(variableNames)
#varNames = [i.decode("utf-8") for i in variableNames] # convert byte string to regular string
varNames = variableNames

formats = ['<i8', '<U12', '<f8', '<f8', '<f8', '<f8','<f8', '<f8', '<f8', '<f8','<U12', '<f8', '<f8', '<f8', '<f8', '<i8']
dtypeNew = np.dtype({'names':dataXYtmp.dtype.names,'formats':formats})
dataXY = dataXYtmp.copy().astype(dtypeNew)
dataXY.dtype.names = varNames

/home/ivanklyuzhin/tensorflow/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['PATNO' 'IMAGEID_BL' 'PUTAMEN_L' 'PUTAMEN_R' 'CAUDATE_L' 'CAUDATE_R'
 'PUTAMEN_B' 'PUTAMEN_W' 'CAUDATE_B' 'CAUDATE_W' 'AGE_BL' 'UPSIT_BL'
 'REM_BL' 'ABETA42_BL' 'COG_V06' 'GENDER']


/home/ivanklyuzhin/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [8]:
# Define data parameters
dataDef = {'imageDir':homeDir + 'ivanklyuzhin/DATA/PPMI/IMAGES4CNN-DYN-NOISE-FLIP/',
              'imageSize':[32, 48, 16],
              'xVarNames':['AGE_BL','UPSIT_BL','REM_BL','ABETA42_BL'], #'AGE_BL','UPSIT_BL','REM_BL','ABETA42_BL'  ||| 'PUTAMEN_L','PUTAMEN_R','CAUDATE_L','CAUDATE_R'
              'yVarName':'COG_V06'}

# Define CNN parameters
cnnDef = {'batchSize':32,
         'iterations':2500}
augsPerClass = 8000
outputDir = homeDir + 'ivanklyuzhin/DATA/PPMI/RESULTS/COG2y-DYN-NOISE-FLIP-META-test-A8000B32I2500/'

numCrossVal = partitions.shape[1]

if os.path.isdir(outputDir)==False:
    os.makedirs(outputDir)
#for n_cv in range(numCrossVal):
for n_cv in range(10):
    print('Computing partition ' + str(n_cv))
    # select partition
    partVec = partitions[:,n_cv]
    # split train and test data
    dataSplit = splitTrainTest(dataXY,partVec)
    dataTrain = dataSplit['trainData']
    dataTest = dataSplit['testData']
    # augment train data
    dataTrainAug = augmentData(dataTrain, outcomeName = 'COG_V06', numberPerClass = augsPerClass)
    # randomize row order
    np.random.seed(0)
    np.random.shuffle(dataTrainAug)
    # Run CNN training and testing
    resultsDict = cnnTrainTest(dataTrainAug,dataTest,dataDef,cnnDef,outputDir)
    resultFilename = outputDir + 'result-cv' + str(n_cv) + '.mat'
    sio.savemat(resultFilename,resultsDict)

Computing partition 0
Total train size 16000, batch size 32, iterations per epoch 500
Training with 2500 iterations, 5.000000 epochs...
Iteration 0, train loss 524.192, test loss 207.379
Class ratio 0.5
Iteration 10, train loss 146.92, test loss 38.8505
Class ratio 0.5625
Iteration 20, train loss 98.7829, test loss 34.581
Class ratio 0.53125
Iteration 30, train loss 29.3449, test loss 140.56
Class ratio 0.5
Iteration 40, train loss 46.2932, test loss 25.5374
Class ratio 0.375
Iteration 50, train loss 64.6205, test loss 12.7393
Class ratio 0.46875
Iteration 60, train loss 64.1794, test loss 22.5827
Class ratio 0.5
Iteration 70, train loss 51.8778, test loss 29.5966
Class ratio 0.34375
Iteration 80, train loss 116.665, test loss 29.5104
Class ratio 0.65625
Iteration 90, train loss 48.8416, test loss 26.187
Class ratio 0.65625
Iteration 100, train loss 68.8883, test loss 32.1685
Class ratio 0.40625
Iteration 110, train loss 101.632, test loss 27.7756
Class ratio 0.59375
Iteration 120, tra

Iteration 1130, train loss 2.32145, test loss 1.73248
Class ratio 0.46875
Iteration 1140, train loss 4.20665, test loss 1.65438
Class ratio 0.6875
Iteration 1150, train loss 3.82913, test loss 1.66601
Class ratio 0.59375
Iteration 1160, train loss 3.1645, test loss 1.72085
Class ratio 0.40625
Iteration 1170, train loss 3.06606, test loss 1.71844
Class ratio 0.5625
Iteration 1180, train loss 3.74207, test loss 1.71883
Class ratio 0.5
Iteration 1190, train loss 3.52262, test loss 1.7238
Class ratio 0.40625
Iteration 1200, train loss 2.27749, test loss 1.73797
Class ratio 0.40625
Iteration 1210, train loss 3.26933, test loss 1.8185
Class ratio 0.375
Iteration 1220, train loss 3.19935, test loss 1.84405
Class ratio 0.40625
Iteration 1230, train loss 3.10913, test loss 1.79637
Class ratio 0.53125
Iteration 1240, train loss 4.21007, test loss 1.78488
Class ratio 0.5625
Iteration 1250, train loss 3.55557, test loss 1.85793
Class ratio 0.46875
Iteration 1260, train loss 2.77308, test loss 1.79

Iteration 2260, train loss 2.56951, test loss 1.68873
Class ratio 0.5
Iteration 2270, train loss 3.02838, test loss 1.67998
Class ratio 0.53125
Iteration 2280, train loss 3.78725, test loss 1.7683
Class ratio 0.53125
Iteration 2290, train loss 2.49733, test loss 1.71485
Class ratio 0.4375
Iteration 2300, train loss 2.8228, test loss 1.7188
Class ratio 0.46875
Iteration 2310, train loss 3.18081, test loss 1.65924
Class ratio 0.4375
Iteration 2320, train loss 2.86657, test loss 1.62446
Class ratio 0.5
Iteration 2330, train loss 2.31531, test loss 1.5997
Class ratio 0.375
Iteration 2340, train loss 2.11983, test loss 1.69413
Class ratio 0.40625
Iteration 2350, train loss 2.80148, test loss 1.83767
Class ratio 0.4375
Iteration 2360, train loss 3.12501, test loss 1.76436
Class ratio 0.46875
Iteration 2370, train loss 1.87147, test loss 1.65776
Class ratio 0.40625
Iteration 2380, train loss 3.01836, test loss 1.64952
Class ratio 0.5
Iteration 2390, train loss 2.87177, test loss 1.59763
Class

Iteration 890, train loss 2.5878, test loss 4.03926
Class ratio 0.40625
Iteration 900, train loss 3.11359, test loss 4.09174
Class ratio 0.46875
Iteration 910, train loss 1.68453, test loss 4.07477
Class ratio 0.375
Iteration 920, train loss 2.14815, test loss 3.98665
Class ratio 0.4375
Iteration 930, train loss 2.53065, test loss 3.82628
Class ratio 0.5
Iteration 940, train loss 2.75879, test loss 3.82175
Class ratio 0.625
Iteration 950, train loss 2.93879, test loss 3.82048
Class ratio 0.59375
Iteration 960, train loss 2.87118, test loss 3.75332
Class ratio 0.40625
Iteration 970, train loss 3.37112, test loss 3.75416
Class ratio 0.625
Iteration 980, train loss 3.89865, test loss 3.89206
Class ratio 0.46875
Iteration 990, train loss 3.81996, test loss 3.77621
Class ratio 0.59375
Iteration 1000, train loss 3.07846, test loss 3.87393
Class ratio 0.5
Iteration 1010, train loss 2.94188, test loss 3.82797
Class ratio 0.5625
Iteration 1020, train loss 2.97187, test loss 3.67882
Class ratio 

Iteration 2020, train loss 2.81979, test loss 3.78656
Class ratio 0.53125
Iteration 2030, train loss 3.19074, test loss 3.84417
Class ratio 0.5
Iteration 2040, train loss 2.35431, test loss 3.95238
Class ratio 0.375
Iteration 2050, train loss 3.25436, test loss 4.08491
Class ratio 0.46875
Iteration 2060, train loss 3.03721, test loss 4.06562
Class ratio 0.5
Iteration 2070, train loss 2.17807, test loss 4.25934
Class ratio 0.34375
Iteration 2080, train loss 2.98207, test loss 4.18125
Class ratio 0.65625
Iteration 2090, train loss 4.83342, test loss 4.03364
Class ratio 0.65625
Iteration 2100, train loss 2.20256, test loss 4.05774
Class ratio 0.40625
Iteration 2110, train loss 3.05268, test loss 4.34409
Class ratio 0.59375
Iteration 2120, train loss 2.1664, test loss 4.45288
Class ratio 0.375
Iteration 2130, train loss 3.57815, test loss 4.35247
Class ratio 0.46875
Iteration 2140, train loss 3.45507, test loss 3.86096
Class ratio 0.6875
Iteration 2150, train loss 3.61009, test loss 3.8490

...

Iteration 320, train loss 3.93886, test loss 2.80135
Class ratio 0.5
Iteration 330, train loss 2.50914, test loss 2.65165
Class ratio 0.375
Iteration 340, train loss 2.76569, test loss 3.45829
Class ratio 0.40625
Iteration 350, train loss 4.72587, test loss 3.79507
Class ratio 0.4375
Iteration 360, train loss 2.82807, test loss 1.99964
Class ratio 0.46875
Iteration 370, train loss 2.86464, test loss 1.81681
Class ratio 0.40625
Iteration 380, train loss 2.73968, test loss 1.81583
Class ratio 0.5
Iteration 390, train loss 1.84263, test loss 1.89044
Class ratio 0.40625
Iteration 400, train loss 3.13904, test loss 1.99453
Class ratio 0.46875
Iteration 410, train loss 2.96924, test loss 1.88718
Class ratio 0.375
Iteration 420, train loss 2.97852, test loss 1.87065
Class ratio 0.4375
Iteration 430, train loss 3.36156, test loss 1.89493
Class ratio 0.5
Iteration 440, train loss 3.43684, test loss 1.91265
Class ratio 0.625
Iteration 450, train loss 3.52441, test loss 1.9999
Class ratio 0.59375

Iteration 1460, train loss 2.53252, test loss 3.61904
Class ratio 0.40625
Iteration 1470, train loss 3.249, test loss 2.60063
Class ratio 0.625
Iteration 1480, train loss 2.5342, test loss 2.64772
Class ratio 0.46875
Iteration 1490, train loss 3.39636, test loss 2.87685
Class ratio 0.59375
Iteration 1500, train loss 3.12277, test loss 1.99074
Class ratio 0.5
Iteration 1510, train loss 3.05817, test loss 1.85201
Class ratio 0.5625
Iteration 1520, train loss 3.65337, test loss 1.96808
Class ratio 0.53125
Iteration 1530, train loss 3.42136, test loss 2.31895
Class ratio 0.5
Iteration 1540, train loss 2.92856, test loss 2.62716
Class ratio 0.375
Iteration 1550, train loss 3.36402, test loss 2.11485
Class ratio 0.46875
Iteration 1560, train loss 2.96026, test loss 2.10287
Class ratio 0.5
Iteration 1570, train loss 2.55126, test loss 2.2286
Class ratio 0.34375
Iteration 1580, train loss 3.18008, test loss 2.15123
Class ratio 0.65625
Iteration 1590, train loss 4.76692, test loss 2.17668
Class

Iteration 80, train loss 32.5081, test loss 13.3738
Class ratio 0.65625
Iteration 90, train loss 105.786, test loss 35.4134
Class ratio 0.65625
Iteration 100, train loss 69.886, test loss 41.7528
Class ratio 0.40625
Iteration 110, train loss 44.079, test loss 19.3099
Class ratio 0.59375
Iteration 120, train loss 3.6169, test loss 2.06462
Class ratio 0.375
Iteration 130, train loss 4.01218, test loss 1.72213
Class ratio 0.46875
Iteration 140, train loss 5.99455, test loss 1.68458
Class ratio 0.6875
Iteration 150, train loss 5.26757, test loss 1.64581
Class ratio 0.59375
Iteration 160, train loss 2.90926, test loss 1.61123
Class ratio 0.40625
Iteration 170, train loss 3.67611, test loss 1.58407
Class ratio 0.5625
Iteration 180, train loss 3.10431, test loss 1.56263
Class ratio 0.5
Iteration 190, train loss 2.60226, test loss 1.54653
Class ratio 0.40625
Iteration 200, train loss 3.09472, test loss 1.53989
Class ratio 0.40625
Iteration 210, train loss 2.70223, test loss 1.54482
Class ratio

Iteration 1220, train loss 2.34095, test loss 1.6314
Class ratio 0.40625
Iteration 1230, train loss 2.50907, test loss 1.5923
Class ratio 0.53125
Iteration 1240, train loss 2.45852, test loss 1.58274
Class ratio 0.5625
Iteration 1250, train loss 2.56024, test loss 1.60995
Class ratio 0.46875
Iteration 1260, train loss 2.83989, test loss 1.67657
Class ratio 0.5
Iteration 1270, train loss 2.91537, test loss 1.69323
Class ratio 0.53125
Iteration 1280, train loss 3.21928, test loss 1.7253
Class ratio 0.53125
Iteration 1290, train loss 3.21791, test loss 1.64573
Class ratio 0.4375
Iteration 1300, train loss 3.05651, test loss 1.6266
Class ratio 0.46875
Iteration 1310, train loss 2.77988, test loss 1.60752
Class ratio 0.4375
Iteration 1320, train loss 2.71909, test loss 1.60805
Class ratio 0.5
Iteration 1330, train loss 2.43223, test loss 1.61859
Class ratio 0.375
Iteration 1340, train loss 2.54846, test loss 1.62801
Class ratio 0.40625
Iteration 1350, train loss 2.73882, test loss 1.65508
C

Iteration 2350, train loss 2.73509, test loss 1.56649
Class ratio 0.4375
Iteration 2360, train loss 2.12626, test loss 1.5858
Class ratio 0.46875
Iteration 2370, train loss 2.48838, test loss 1.64017
Class ratio 0.40625
Iteration 2380, train loss 2.38795, test loss 1.62845
Class ratio 0.5
Iteration 2390, train loss 2.31198, test loss 1.62938
Class ratio 0.40625
Iteration 2400, train loss 2.4276, test loss 1.51657
Class ratio 0.46875
Iteration 2410, train loss 2.03097, test loss 1.50602
Class ratio 0.375
Iteration 2420, train loss 2.6332, test loss 1.52956
Class ratio 0.4375
Iteration 2430, train loss 2.73663, test loss 1.54225
Class ratio 0.5
Iteration 2440, train loss 2.29446, test loss 1.52174
Class ratio 0.625
Iteration 2450, train loss 2.82233, test loss 1.5309
Class ratio 0.59375
Iteration 2460, train loss 2.15778, test loss 1.53238
Class ratio 0.40625
Iteration 2470, train loss 2.9059, test loss 1.49685
Class ratio 0.625
Iteration 2480, train loss 2.73244, test loss 1.53249
Class

In [9]:
dataFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/data-cog4y.csv'
variableFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/names-cog4y.csv'
partitionsFilename = homeDir + 'ivanklyuzhin/Dropbox/CODE/ANALYSIS/PPMI/res/part-cog4y.csv'

dataXYtmp = readDataFromCsv(dataFilename)
variableNames = readDataFromCsv(variableFilename)
partitions = readDataFromCsv(partitionsFilename) # 1's are for train, 2's are for test

print(variableNames)
#varNames = [i.decode("utf-8") for i in variableNames] # convert byte string to regular string
varNames = variableNames

formats = ['<i8', '<U12', '<f8', '<f8', '<f8', '<f8','<f8', '<f8', '<f8', '<f8','<U12', '<f8', '<f8', '<f8', '<f8', '<i8']
dtypeNew = np.dtype({'names':dataXYtmp.dtype.names,'formats':formats})
dataXY = dataXYtmp.copy().astype(dtypeNew)
dataXY.dtype.names = varNames

['PATNO' 'IMAGEID_BL' 'PUTAMEN_L' 'PUTAMEN_R' 'CAUDATE_L' 'CAUDATE_R'
 'PUTAMEN_B' 'PUTAMEN_W' 'CAUDATE_B' 'CAUDATE_W' 'AGE_BL' 'UPSIT_BL'
 'REM_BL' 'ABETA42_BL' 'COG_V10' 'GENDER']


/home/ivanklyuzhin/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [10]:
# Define data parameters
dataDef = {'imageDir':homeDir + 'ivanklyuzhin/DATA/PPMI/IMAGES4CNN-DYN-NOISE-FLIP/',
              'imageSize':[32, 48, 16],
              'xVarNames':['AGE_BL','UPSIT_BL','REM_BL','ABETA42_BL'], #'AGE_BL','UPSIT_BL','REM_BL','ABETA42_BL'  ||| 'PUTAMEN_L','PUTAMEN_R','CAUDATE_L','CAUDATE_R'
              'yVarName':'COG_V10'}
# Define CNN parameters
cnnDef = {'batchSize':32,
         'iterations':2500}
augsPerClass = 8000
outputDir = homeDir + 'ivanklyuzhin/DATA/PPMI/RESULTS/COG4y-DYN-NOISE-FLIP-META-test-A8000B32I2500/'

numCrossVal = partitions.shape[1]

if os.path.isdir(outputDir)==False:
    os.makedirs(outputDir)
#for n_cv in range(numCrossVal):
for n_cv in range(10):
    print('Computing partition ' + str(n_cv))
    # select partition
    partVec = partitions[:,n_cv]
    # split train and test data
    dataSplit = splitTrainTest(dataXY,partVec)
    dataTrain = dataSplit['trainData']
    dataTest = dataSplit['testData']
    # augment train data
    dataTrainAug = augmentData(dataTrain, outcomeName = 'COG_V10', numberPerClass = augsPerClass)
    # randomize row order
    np.random.seed(0)
    np.random.shuffle(dataTrainAug)
    # Run CNN training and testing
    resultsDict = cnnTrainTest(dataTrainAug,dataTest,dataDef,cnnDef,outputDir)
    resultFilename = outputDir + 'result-cv' + str(n_cv) + '.mat'
    sio.savemat(resultFilename,resultsDict)

Computing partition 0
Total train size 16000, batch size 32, iterations per epoch 500
Training with 2500 iterations, 5.000000 epochs...
Iteration 0, train loss 459.529, test loss 261.097
Class ratio 0.5
Iteration 10, train loss 80.1332, test loss 32.0862
Class ratio 0.5625
Iteration 20, train loss 33.5345, test loss 54.7118
Class ratio 0.53125
Iteration 30, train loss 77.7416, test loss 43.5305
Class ratio 0.5
Iteration 40, train loss 47.2124, test loss 28.1634
Class ratio 0.375
Iteration 50, train loss 63.2682, test loss 120.775
Class ratio 0.46875
Iteration 60, train loss 54.0482, test loss 25.859
Class ratio 0.5
Iteration 70, train loss 55.2857, test loss 37.2937
Class ratio 0.34375
Iteration 80, train loss 134.909, test loss 50.2941
Class ratio 0.65625
Iteration 90, train loss 67.2434, test loss 20.569
Class ratio 0.65625
Iteration 100, train loss 56.542, test loss 34.3236
Class ratio 0.40625
Iteration 110, train loss 88.9211, test loss 40.299
Class ratio 0.59375
Iteration 120, tra

Iteration 1130, train loss 3.39611, test loss 1.94925
Class ratio 0.46875
Iteration 1140, train loss 4.34948, test loss 1.87854
Class ratio 0.6875
Iteration 1150, train loss 4.81886, test loss 1.83969
Class ratio 0.59375
Iteration 1160, train loss 3.50289, test loss 1.82496
Class ratio 0.40625
Iteration 1170, train loss 3.26065, test loss 1.80874
Class ratio 0.5625
Iteration 1180, train loss 3.21524, test loss 1.80997
Class ratio 0.5
Iteration 1190, train loss 3.07165, test loss 1.82659
Class ratio 0.40625
Iteration 1200, train loss 3.08768, test loss 1.86079
Class ratio 0.40625
Iteration 1210, train loss 2.02152, test loss 1.88237
Class ratio 0.375
Iteration 1220, train loss 3.30931, test loss 1.89485
Class ratio 0.40625
Iteration 1230, train loss 3.34386, test loss 1.8217
Class ratio 0.53125
Iteration 1240, train loss 3.41202, test loss 1.80798
Class ratio 0.5625
Iteration 1250, train loss 2.85205, test loss 1.83785
Class ratio 0.46875
Iteration 1260, train loss 4.08958, test loss 1.

Iteration 2260, train loss 3.46752, test loss 1.99212
Class ratio 0.5
Iteration 2270, train loss 2.74803, test loss 2.00329
Class ratio 0.53125
Iteration 2280, train loss 3.06275, test loss 1.97497
Class ratio 0.53125
Iteration 2290, train loss 2.70853, test loss 1.90535
Class ratio 0.4375
Iteration 2300, train loss 2.48659, test loss 1.90519
Class ratio 0.46875
Iteration 2310, train loss 2.413, test loss 1.91289
Class ratio 0.4375
Iteration 2320, train loss 2.47032, test loss 1.91881
Class ratio 0.5
Iteration 2330, train loss 2.76204, test loss 1.94388
Class ratio 0.375
Iteration 2340, train loss 2.7627, test loss 1.99089
Class ratio 0.40625
Iteration 2350, train loss 3.06603, test loss 2.09247
Class ratio 0.4375
Iteration 2360, train loss 2.32449, test loss 2.00212
Class ratio 0.46875
Iteration 2370, train loss 2.60118, test loss 1.92632
Class ratio 0.40625
Iteration 2380, train loss 2.87996, test loss 1.90908
Class ratio 0.5
Iteration 2390, train loss 2.5985, test loss 1.91587
Class

Iteration 890, train loss 3.36562, test loss 1.99961
Class ratio 0.40625
Iteration 900, train loss 3.98913, test loss 1.92612
Class ratio 0.46875
Iteration 910, train loss 2.23253, test loss 1.9117
Class ratio 0.375
Iteration 920, train loss 3.56556, test loss 1.89615
Class ratio 0.4375
Iteration 930, train loss 3.42883, test loss 1.87464
Class ratio 0.5
Iteration 940, train loss 4.07665, test loss 1.84907
Class ratio 0.625
Iteration 950, train loss 4.31624, test loss 1.83373
Class ratio 0.59375
Iteration 960, train loss 2.25308, test loss 1.82952
Class ratio 0.40625
Iteration 970, train loss 3.46199, test loss 1.82321
Class ratio 0.625
Iteration 980, train loss 3.64673, test loss 1.84325
Class ratio 0.46875
Iteration 990, train loss 3.34614, test loss 1.85991
Class ratio 0.59375
Iteration 1000, train loss 2.70959, test loss 1.90745
Class ratio 0.5
Iteration 1010, train loss 3.22786, test loss 1.90396
Class ratio 0.5625
Iteration 1020, train loss 3.52453, test loss 1.8977
Class ratio 0

Iteration 2020, train loss 3.21192, test loss 1.78478
Class ratio 0.53125
Iteration 2030, train loss 2.58477, test loss 1.82422
Class ratio 0.5
Iteration 2040, train loss 2.27421, test loss 1.85202
Class ratio 0.375
Iteration 2050, train loss 2.96167, test loss 1.79559
Class ratio 0.46875
Iteration 2060, train loss 3.16533, test loss 1.81279
Class ratio 0.5
Iteration 2070, train loss 2.11894, test loss 1.83977
Class ratio 0.34375
Iteration 2080, train loss 3.61419, test loss 1.84364
Class ratio 0.65625
Iteration 2090, train loss 4.25819, test loss 1.8107
Class ratio 0.65625
Iteration 2100, train loss 2.52098, test loss 1.82071
Class ratio 0.40625
Iteration 2110, train loss 3.1435, test loss 1.86392
Class ratio 0.59375
Iteration 2120, train loss 2.52707, test loss 1.89877
Class ratio 0.375
Iteration 2130, train loss 2.94545, test loss 1.88088
Class ratio 0.46875
Iteration 2140, train loss 3.95083, test loss 1.73469
Class ratio 0.6875
Iteration 2150, train loss 3.32029, test loss 1.72421

...

Iteration 320, train loss 3.99973, test loss 1.79249
Class ratio 0.5
Iteration 330, train loss 3.04122, test loss 1.81357
Class ratio 0.375
Iteration 340, train loss 2.89761, test loss 1.82746
Class ratio 0.40625
Iteration 350, train loss 4.46733, test loss 1.88179
Class ratio 0.4375
Iteration 360, train loss 2.72255, test loss 1.87056
Class ratio 0.46875
Iteration 370, train loss 4.69656, test loss 1.90081
Class ratio 0.40625
Iteration 380, train loss 2.9036, test loss 1.86046
Class ratio 0.5
Iteration 390, train loss 4.10238, test loss 1.8156
Class ratio 0.40625
Iteration 400, train loss 3.60018, test loss 1.8053
Class ratio 0.46875
Iteration 410, train loss 3.25322, test loss 1.78857
Class ratio 0.375
Iteration 420, train loss 2.61983, test loss 1.75851
Class ratio 0.4375
Iteration 430, train loss 3.71888, test loss 1.76244
Class ratio 0.5
Iteration 440, train loss 3.86485, test loss 1.76934
Class ratio 0.625
Iteration 450, train loss 4.12674, test loss 1.76873
Class ratio 0.59375
I

Iteration 1460, train loss 2.73258, test loss 1.7193
Class ratio 0.40625
Iteration 1470, train loss 4.0801, test loss 1.71671
Class ratio 0.625
Iteration 1480, train loss 2.87233, test loss 1.76218
Class ratio 0.46875
Iteration 1490, train loss 3.49119, test loss 1.77646
Class ratio 0.59375
Iteration 1500, train loss 3.07241, test loss 1.7993
Class ratio 0.5
Iteration 1510, train loss 3.29562, test loss 1.77981
Class ratio 0.5625
Iteration 1520, train loss 3.39064, test loss 1.72151
Class ratio 0.53125
Iteration 1530, train loss 2.99801, test loss 1.70237
Class ratio 0.5
Iteration 1540, train loss 2.80983, test loss 1.74315
Class ratio 0.375
Iteration 1550, train loss 3.01457, test loss 1.75841
Class ratio 0.46875
Iteration 1560, train loss 3.12685, test loss 1.7538
Class ratio 0.5
Iteration 1570, train loss 2.826, test loss 1.77128
Class ratio 0.34375
Iteration 1580, train loss 3.50517, test loss 1.77524
Class ratio 0.65625
Iteration 1590, train loss 4.56474, test loss 1.78649
Class r

Iteration 80, train loss 44.9074, test loss 33.8102
Class ratio 0.65625
Iteration 90, train loss 193.812, test loss 86.2461
Class ratio 0.65625
Iteration 100, train loss 118.718, test loss 74.2299
Class ratio 0.40625
Iteration 110, train loss 24.5884, test loss 37.0365
Class ratio 0.59375
Iteration 120, train loss 63.2227, test loss 40.9681
Class ratio 0.375
Iteration 130, train loss 37.8128, test loss 17.4385
Class ratio 0.46875
Iteration 140, train loss 83.0312, test loss 30.4304
Class ratio 0.6875
Iteration 150, train loss 34.1291, test loss 94.6154
Class ratio 0.59375
Iteration 160, train loss 87.8862, test loss 24.0596
Class ratio 0.40625
Iteration 170, train loss 65.7097, test loss 33.7698
Class ratio 0.5625
Iteration 180, train loss 32.8416, test loss 27.4259
Class ratio 0.5
Iteration 190, train loss 41.5918, test loss 17.7791
Class ratio 0.40625
Iteration 200, train loss 71.0136, test loss 42.6576
Class ratio 0.40625
Iteration 210, train loss 50.2245, test loss 43.1033
Class ra

Iteration 1220, train loss 5.44443, test loss 9.83714
Class ratio 0.40625
Iteration 1230, train loss 16.2516, test loss 10.1124
Class ratio 0.53125
Iteration 1240, train loss 20.2925, test loss 10.9158
Class ratio 0.5625
Iteration 1250, train loss 5.59275, test loss 12.6769
Class ratio 0.46875
Iteration 1260, train loss 16.6125, test loss 8.77301
Class ratio 0.5
Iteration 1270, train loss 11.6008, test loss 8.1605
Class ratio 0.53125
Iteration 1280, train loss 12.2185, test loss 6.7708
Class ratio 0.53125
Iteration 1290, train loss 13.5395, test loss 8.69778
Class ratio 0.4375
Iteration 1300, train loss 8.37365, test loss 6.39043
Class ratio 0.46875
Iteration 1310, train loss 7.91066, test loss 6.29448
Class ratio 0.4375
Iteration 1320, train loss 10.5333, test loss 6.08811
Class ratio 0.5
Iteration 1330, train loss 13.8411, test loss 7.11913
Class ratio 0.375
Iteration 1340, train loss 7.9471, test loss 27.3639
Class ratio 0.40625
Iteration 1350, train loss 9.74988, test loss 7.72013


Iteration 2350, train loss 3.11543, test loss 16.0133
Class ratio 0.4375
Iteration 2360, train loss 4.28253, test loss 20.8547
Class ratio 0.46875
Iteration 2370, train loss 3.08855, test loss 20.6295
Class ratio 0.40625
Iteration 2380, train loss 6.19892, test loss 14.551
Class ratio 0.5
Iteration 2390, train loss 3.27287, test loss 19.0865
Class ratio 0.40625
Iteration 2400, train loss 3.57273, test loss 41.3208
Class ratio 0.46875
Iteration 2410, train loss 9.54613, test loss 7.12467
Class ratio 0.375
Iteration 2420, train loss 11.8727, test loss 7.42586
Class ratio 0.4375
Iteration 2430, train loss 11.4305, test loss 5.8572
Class ratio 0.5
Iteration 2440, train loss 9.21434, test loss 6.44768
Class ratio 0.625
Iteration 2450, train loss 2.63565, test loss 12.4676
Class ratio 0.59375
Iteration 2460, train loss 4.6284, test loss 2.84058
Class ratio 0.40625
Iteration 2470, train loss 5.32164, test loss 2.67222
Class ratio 0.625
Iteration 2480, train loss 3.36543, test loss 2.17011
Cla